In [107]:
# load data and write out sentence and target
import pandas as pd

loaded_set = pd.read_excel("Dataset/"+"training.xlsx")

#x_train, x_test,  y_train, y_test = train_test_split(loaded_set['Sentence'], loaded_set['MOS'], test_size=.2)
loaded_set['Sentence']

0      Weitergehende Sicherungsmaßnahmen können eine ...
1      In der großen Kasernenanlage im Norden Kiels k...
2      Premierminister David Lloyd George honorierte ...
3      Der Beitrag der Truppen dieser Dominions währe...
4      Eine Balance zwischen verschiedenen Lebensbere...
                             ...                        
895    Zwischen 1901 und 2010 ist er um ca 1,7 cm pro...
896    Aus Furcht vor einem Bürgerkrieg wollte sie – ...
897    In den meisten Politikfeldern gilt dafür seit ...
898    Aufgrund der Wärmekapazität des Gesteins, und ...
899    Die Klinge ist zumeist aus nicht rostfreiem Ko...
Name: Sentence, Length: 900, dtype: object

In [108]:
tokens_num=[]
for sen in loaded_set['Sentence']:
    tokenized = (tokenizer.tokenize(sen)) 
    tokens_num.append( ['[CLS]'] + tokenized + ['[SEP]']) 
    
# get max_seq length    
lens = [len(i) for i in tokens_num]
max_seq_length = max(lens)
max_seq_length = int(1.5*max_seq_length)
max_seq_length

144

In [109]:
import tensorflow_hub as hub
from transformers import AutoModel, AutoTokenizer
# german tokens for bert
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-german-cased")
#model = AutoModel.from_pretrained("dbmdz/bert-base-german-cased")

In [110]:
import numpy as np
def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [121]:
x = loaded_set['Sentence']
y = loaded_set['MOS']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=32)
y_train = round(y_train, 2)
y_test = round(y_test, 2)
y_train

49     2.30
863    2.11
607    4.07
853    2.62
74     3.29
       ... 
88     1.89
892    1.31
310    3.20
555    2.11
727    4.00
Name: MOS, Length: 720, dtype: float64

In [122]:

X_train = bert_encode(x_train, tokenizer, max_seq_length)
X_test = bert_encode(x_test, tokenizer, max_seq_length)


In [135]:
# set up a 1d CNN
import tensorflow as tf
num_class = 1  # Based on available class selection
max_seq_length = max_seq_length  # we calculated this a couple cells ago

input_word_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                       name="input_word_ids")
input_mask = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                   name="input_mask")
segment_ids = tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32,
                                    name="segment_ids")

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_multi_cased_L-12_H-768_A-12/1",
                            trainable=True)
pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

output_drop = tf.keras.layers.Dropout(rate=0.1)(sequence_output)
output_dense_1 = tf.keras.layers.Dense(20, activation='relu', name='dense_1')(output_drop)
output = tf.keras.layers.Dense(1, activation='softmax', name='output')(output_dense_1)

model = tf.keras.Model(
    inputs={
        'input_word_ids':input_word_ids,
        'input_mask':input_mask,
        'input_type_ids':segment_ids
    },
    outputs = output
)

In [136]:


epochs = 3
batch_size = 8  # select based on your GPU resources
eval_batch_size = batch_size

train_data_size = len(y_train)
steps_per_epoch = int(train_data_size / batch_size)
num_train_steps = steps_per_epoch * epochs
warmup_steps = int(epochs * train_data_size * 0.1 / batch_size)

optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam'
)




In [141]:
model.compile(optimizer=optimizer,
            loss='mse',
            metrics=['accuracy']
            )

In [142]:
model.summary()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 144)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 144)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 144)]        0                                            
__________________________________________________________________________________________________
keras_layer_7 (KerasLayer)      [(None, 768), (None, 177853441   input_word_ids[0][0]             
                                                                 input_mask[0][0]           

In [143]:
history = model.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test, y_test), verbose=1)

90/90 [==============================] - 637s 7s/step - loss: 5.5403 - accuracy: 0.0611 - val_loss: 5.0111 - val_accuracy: 0.1111
